In [1]:
import json
import logging
import pandas as pd
import numpy as np
import os
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy import get_logger
from yfpy.utils import unpack_data, dict_to_list, complex_json_handler
from dotenv import load_dotenv
from gspread_pandas import Spread

pd.set_option("display.max_colwidth", 999)

from utils import get_season, league_season_info, google_sheet_upload, sql_upload_table


load_dotenv()
LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = Path.cwd().parents[0]
DATA_DIR = PATH / "data"

SEASON = get_season()
NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
NFL_WEEK = NFL_DATES_DF["Week"][
    (NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)
].values[0]
LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
WEEKS = list(range(NFL_WEEK, 0, -1))

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

try:
    yahoo_query = YahooFantasySportsQuery(
        auth_dir=PATH,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )

except Exception as e:
    print(e)

In [2]:
query = yahoo_query.get_all_yahoo_fantasy_game_keys()
data = unpack_data(query)
df = pd.DataFrame()
for row in data:
    row = complex_json_handler(row['game'])
    r = pd.DataFrame(row, index=[0])
    df = pd.concat([df, r])
df.reset_index(drop=True, inplace=True)
sql_upload_table(dataframe=df, table_name="GameKeys", data_schema="dbo", chunksize=500, if_exists="replace", index=False)

[2022-06-02 16:06:06,712 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <pyodbc.Connection object at 0x000002277FF9AA70>
[2022-06-02 16:06:06,712 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <pyodbc.Connection object at 0x000002277FF9AA70>
[2022-06-02 16:06:06,713 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
[2022-06-02 16:06:06,713 INFO] [sqlalchemy.engine.Engine.base._execute_context] SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
[2022-06-02 16:06:06,714 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] ()
[2022-06-02 16:06:06,714 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] ()
[2022-06-02 16:06:06,715 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-06-02 16:06:06,715 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('',)
[2022-06-02 16:06:06,716 DEBUG] [sqlalchemy.engine.Eng

GameKeys successfully updated or added to SQL server.
